In this notebook I'll briefly show how to simply automate any transaction queries in SAP R/3 (or most of them anyway...) with a _hybrid solution_ with Python and VB Scripting

SAP R/3 has a built-in recorder that is just like a MS Excel Macro Recorder. It records every action you would do in a session and write a script with everything that has been done and this will be essential for our tool.

As an example I'll automate a FBL3N report extraction. It consists in:
- Setting a list accounts;
- Setting a list of companies;
- Setting a time range;
- Choosing a Layout;
- Executing the transaction;
- Exporting the report for a local file (.xls);

All these actions can be recorded with the SAP GUI Recorder and made into vbs scripts. You can check the vbs_scripts folder for those files, but it's probabbly best if you record yours yourself since sometimes these ERP systems are build differently.

Some input info like the 'list of accounts' and 'list of companies' are very dense lists, containing hundreds of items, so it is best if you build a dataframe containing these information you want to use as query parameter.

SAP also works with _copy and paste_ (good'ol ctrl+c/ctrl+v), and you can gain a lot of speed by using these commands in your recordings. So i'll present a function that copies a column of a dataframe to your cliboard, so your vbs scripts can paste it in SAP.

In [4]:
import re
import pyperclip

def dfcol_to_clipboard(dataframe, column_name):
    copied_string = dataframe[column_name].to_string(index = False)
    copied_string = re.sub(r"\s{2,}","\r\n", copied_sring)
    pyperclip.copy(copied_string)
    return 'done!'

Now, lets just suppose you have a pandas script that reads a spreadsheet with collumns of accounts and companies that you want to explore in FBL3N. And this scripts spits off a dataframe that contain dozens of columns but with 'accounts' and 'companies' being two of it. It will be argument to the FBL3N function.

The function below will execute the sequence of vbs scripts in order to automate the FBL3N query in SAP, provided you have given it a data frame to copy those accounts and companies from.

In [9]:
import pandas as pd
import numpy as np
import easygui as ezg
import subprocess
import os

def FBL3N(dataframe, outName = 'FBL3N.xls'):
    
    #very basic UI, but you can use whatever you want to get this information
    
    vbsFolder = ezg.diropenbox(msg = 'select your vbs scripts folder')
    outputFolder = ezg.diropenbox(msg = 'select your output folder')
    start_date = ezg.enterbox(msg = 'enter start date as DD.MM.YYYY')
    end_date = ezg.enterbox(msg = 'enter final date as DD.MM.YYYY')

    #Executing the sequence of vbs scripts
    
    #Runs the script that start FBL3N
    vbs_fbl3n_start = os.path.join(vbsFolder, 'fbl3n_start.vbs')
    subprocess.call(['cscript.exe', vbs_fbl3n_start])
    
    #Runs the script to input accounts
    vbs_fbl3n_account = os.path.join(vbsFolder, 'fbl3n_account.vbs')
    dfcolumn_to_clipboard(dataframe, 'account')
    subprocess.call(['cscript.exe', vbs_fbl3n_account])
    
    #Runs the script o input companies
    vbs_fbl3n_company = os.path.join(vbsFolder, 'fbl3n_company.vbs')
    dfcolumn_to_clipboard(dataframe, 'company')
    subprocess.call(['cscript.exe', vbs_fbl3n_company])
    
    #Runs the script to input dates
    vbs_fbl3n_dates = os.path.join(vbsFolder, 'fbl3n_dates.vbs')
    subprocess.call(['cscript.exe', vbs_fbl3n_dates, start_date, end_date]) #here, star_date and end_date becomes WScript.Arguments(0) and (1) respectively
    
    #Runs the script that chooses layout
    vbs_fbl3n_layout = os.path.join(vbsFolder, 'fbl3n_layout.vbs')
    subprocess.call(['cscript.exe', vbs_fbl3n_layout])
    
    #Runs the script that executes the tranaction
    vbs_fbl3n_execute = os.path.join(vbsFolder, 'fbl3n_execute.vbs')
    subprocess.call(['cscript.exe', vbs_fbl3n_execute])
    
    #Runs the extraction of the result in a .xls file
    vbs_fbl3n_extract = os.path.join(vbsFolder, 'fbl3n_extract.vbs')
    output_path = os.path.join(outputFolder, outName)
    if os.path.exists(output_path):
        os.remove(output_path)   
    subprocess.call(['cscript.exe', vbs_fbl3n_extract, outputFolder, outName])
    

The essence of automating chores that way is basically:

- Record the tasks in SAP GUI Recorder;
- Segregate them to help you keeping thigs clean and modular;
- Build the python code according to this;

On my work I also do a lot of pre-processing in the input dataframe and even convert/parse the output file in a pandas dataframe for further work with the data, but they contain sensible stuff, so I've skipped it.